In [19]:
from utils import *
import json
import random
import pandas as pd
import numpy as np
import torch as th


class Args:
    data_frac = 0.05
    min_user_freq = 10
    min_book_freq = 10
    max_user_freq = 200
    train_frac = 0.95
    his_len = 100
    n_neg = 10


args = Args()


In [20]:
valid_users, valid_books = get_valid_user_and_book_set(args)
all_user_rates = parse_rating_data(valid_users, valid_books)

In [23]:
def get_user_rating_size(all_user_rates):
    all_user_rating_size = {}
    for user_id, ratings in all_user_rates.items():
        all_user_rating_size[user_id] = len(ratings)

    return all_user_rating_size

all_user_rating_size = get_user_rating_size(all_user_rates)


In [24]:
all_user_rating_size

{'A2S166WSCFIFP5': 140,
 'AWLFVCT9128JV': 65,
 'AMRZ5G7HF7I03': 29,
 'A2NHD7LUXVGTD3': 201,
 'A1K1JW1C5CUSUZ': 689,
 'ADS5APY1NKTL4': 17,
 'A38AAPXSJN4C5G': 81,
 'A2BQZRA2P81BQG': 13,
 'A3UJJGY799F76I': 68,
 'A2OP1HD9RGX5OW': 46,
 'A2X6GEC6LCDN4S': 27,
 'A1FQPOYRBTTK1': 153,
 'A7GT0WQKKDP0V': 32,
 'AGKPTMTR3UX1R': 58,
 'A1P5UBFPQ48OWN': 51,
 'APHX2A37KDRAH': 34,
 'ANTGGAQYPSWHF': 111,
 'AJRFZ0VZ0LD26': 55,
 'A281NPSIMI1C2R': 161,
 'A6HZ4G4G5RWNK': 46,
 'AMKZHBOK7VMQR': 65,
 'ANRE1BJ0Q4PLS': 27,
 'ALDRY40BPNY09': 77,
 'AHD101501WCN1': 231,
 'A1XJOSJN6FHFO0': 29,
 'A3R8PXSFGY9MC2': 98,
 'A2UTKYA2U8542F': 69,
 'A27XUU2DXILHYZ': 35,
 'A1DYXCF4148PJT': 130,
 'APBA7Y4SINS0H': 107,
 'A22019GDU7OVFG': 126,
 'A1HRYC60VTMYC0': 61,
 'A3T4KN9XO3PIXG': 121,
 'A914TQVHI872U': 129,
 'A3E02KBFI21TY0': 142,
 'A2TXMYYGSZCSZ1': 25,
 'A3BIWTN2DA0YY2': 161,
 'A30H2335OM7RD6': 110,
 'A3HKZA48V9HMCA': 20,
 'A21NVBFIEQWDSG': 246,
 'AJQ1S39GZBKUG': 169,
 'A2PN65B6BSTIYZ': 132,
 'A3KOO6L0NA2PJA': 57,
 'A3GUK4RW

In [28]:
test_samples = pkl_read('data/test_samples.pkl')

In [29]:
class Dataset:
    def __init__(self, samples):
        self.samples = samples
        self.users, self.histories, self.tars, self.labels, self.cap_nums = [], [], [], [], []
        for sample in samples:
            self.users.append(sample['user'])
            self.histories.append(sample['his'])
            self.tars.append(sample['tar'])
            self.labels.append(sample['label'])
            self.cap_nums.append(sample['cap_num'])
        self.users = th.tensor(self.users, dtype=th.int32)
        self.histories = th.tensor(self.histories, dtype=th.int32)
        self.tars = th.tensor(self.tars, dtype=th.int32)
        self.labels = th.tensor(self.labels, dtype=th.float)
        self.cap_nums = th.tensor(self.cap_nums, dtype=th.int32)
    
    def __getitem__(self, idx):
        return self.users[idx], self.histories[idx], self.tars[idx], self.labels[idx], self.cap_nums[idx]
    
    def __len__(self):
        return len(self.samples)

test_dataset = Dataset(test_samples)

test_dataloader = \
    th.utils.data.DataLoader(test_dataset, batch_size=2, shuffle=True)


In [40]:

class Capsule()

class MIND(th.nn.Module):
    def __init__(self, args, n_users, n_books, n_book_cates):
        self.dim = args.dim
        self.user_embeds = th.nn.Embedding(n_users, args.dim)
        self.book_embeds = th.nn.Embedding(n_books, args.dim)
        self.book_tag_embeds = th.nn.Embedding(n_book_cates, args.dim)


    def forward(self, batch):
        user_id, history, tars, labels, num_caps = batch[0], batch[1], batch[2], batch[3], batch[4]

torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])
